In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('logs.csv')
df.head()

,Algoritmo,Número de Clusters,Número Máximo de Iterações,Centros dos Clusters,Inércia,Coesão,Coeficiente de Silhueta,Rand Score,Homogeneidade,Completude,Entropia,Matriz de Contingência
0,KMEANS,2.0,100,"[[1.3339195427566404, 1.05869421850957, 2.1200...",199292.488867,223.210937,0.275966,0.729135,0.364306,0.367773,8.182,"[[3722, 486], [826, 3090]]"
1,KMEANS,2.0,200,"[[1.3339195427566404, 1.05869421850957, 2.1200...",199292.488867,223.210937,0.275966,0.729135,0.364306,0.367773,8.182,"[[3722, 486], [826, 3090]]"
2,KMEANS,2.0,300,"[[1.3339195427566404, 1.05869421850957, 2.1200...",199292.488867,223.210937,0.275966,0.729135,0.364306,0.367773,8.182,"[[3722, 486], [826, 3090]]"
3,KMEANS,2.0,400,"[[1.3339195427566404, 1.05869421850957, 2.1200...",199292.488867,223.210937,0.275966,0.729135,0.364306,0.367773,8.182,"[[3722, 486], [826, 3090]]"
4,KMEANS,2.0,500,"[[1.3339195427566404, 1.05869421850957, 2.1200...",199292.488867,223.210937,0.275966,0.729135,0.364306,0.367773,8.182,"[[3722, 486], [826, 3090]]"


In [12]:
df_kmeans = df[df['Algoritmo'] == 'KMEANS']

maior_homogeneidade = df_kmeans['Homogeneidade'].idxmax()
maior_homogeneidade = df_kmeans.iloc[maior_homogeneidade]

maior_completude = df_kmeans['Completude'].idxmax()
maior_completude = df_kmeans.iloc[maior_completude]

maior_completude
# maior_homogeneidade


Algoritmo                                                                KMEANS
Número de Clusters                                                          2.0
Número Máximo de Iterações                                                  100
Centros dos Clusters          [[1.3339195427566404, 1.05869421850957, 2.1200...
Inércia                                                           199292.488867
Coesão                                                               223.210937
Coeficiente de Silhueta                                                0.275966
Rand Score                                                             0.729135
Homogeneidade                                                          0.364306
Completude                                                             0.367773
Entropia                                                                  8.182
Matriz de Contingência                               [[3722, 486], [826, 3090]]
Name: 0, dtype: object